## Логическая модель данных:

<img src="https://github.com/timoti1/T-SQL/blob/master/SQL/img/SwimmersDB.png?raw=1" />

_Хранимые процедуры_ похожи на процедуры из других языков программирования в том, что они могут:

- принимать входные параметры и возвращать вызывающей процедуре или пакету ряд значений в виде выходных параметров;

- содержать программные инструкции, которые выполняют операции в базе данных, в том числе вызывающие другие процедуры;

- возвращать значение состояния вызывающей процедуре или пакету, таким образом передавая сведения об успешном или неуспешном завершении (и причины последнего).

Синтаксис:
```
-- Transact-SQL Syntax for Stored Procedures in SQL Server and Azure SQL Database  
  
CREATE [ OR ALTER ] { PROC | PROCEDURE } 
    [schema_name.] procedure_name [ ; number ]   
    [ { @parameter [ type_schema_name. ] data_type }  
        [ VARYING ] [ = default ] [ OUT | OUTPUT | [READONLY]  
    ] [ ,...n ]   
[ WITH <procedure_option> [ ,...n ] ]  
[ FOR REPLICATION ]   
AS { [ BEGIN ] sql_statement [;] [ ...n ] [ END ] }  
[;]  
  
<procedure_option> ::=   
    [ ENCRYPTION ]  
    [ RECOMPILE ]  
    [ EXECUTE AS Clause ]
```
Подробнее о хранимых процедурах T-SQL:<br/>
https://docs.microsoft.com/ru-ru/sql/t-sql/statements/create-procedure-transact-sql?view=sql-server-2017

Пример #1:

Хранимая процедура, с json-параметром по умолчанию:

In [3]:
use tempdb
go

if OBJECT_ID('dbo.usp_GetSwimmersList', 'P') is not null
   drop procedure dbo.usp_GetSwimmersList
go

-------------------------------------------------------------------------------------------
-- procedure returns list of Swimmers from given clubs
-- created by:   Timofey Gavrilenko
-- created date: 4/22/2019
-- sample call:  
-- exec dbo.usp_GetSwimmersList @parameters = N'[{"Club": "Трактор", "City": "Минск"}]'
-----------------------------------------------------------------------------------

create procedure dbo.usp_GetSwimmersList
    @parameters nvarchar(1000) = null
as    
begin
    select  s.FirstName,
            s.LastName,
            s.YearOfBirth,
            ISNULL(sc.[Name] + ' ' + sc.City, '-') Club,
            ISNULL(c.[Name], '-') Category
    from dbo.Swimmer s
    left join dbo.SwimmingClub sc   on s.SwimmingClubID = sc.SwimmingClubID
    left join dbo.Category c        on s.CategoryID = c.CategoryID
    outer apply (
        select  [Name], 
				City
        from openjson(@parameters) 
            with (
                     [Name] nvarchar(100) '$.Club',
                     City nvarchar(100) '$.City'
                 ) j
        where (j.[Name] = sc.[Name] or j.[Name] is null) and
              (j.City = sc.City or j.City is null)
    ) f
    where (f.City is not null or f.[Name] is not null) or @parameters is null
end
go

_Скалярная функция_ - функция, возвращающая скалярное значение. Как правило, это значение одного из стандартных T-SQL типов (int, varchar(x), datetime и т.д.)

Однако, возвращаемым значением может быть XML, JSON или значение, определенного пользователем типа.<br/>
Значения по умолчанию возможны, но опустить фактический параметр при вызове функции нельзя.

Синтаксис:
```
CREATE  FUNCTION [ owner_name. ] function_name 
    ( [ { @parameter_name [AS] scalar_parameter_data_type [ = default ] } 
      [ ,...n ] ] ) 

RETURNS scalar_return_data_type

[ WITH function_option [ [,] ...n] ] 

[ AS ]

BEGIN 
    function_body 
    RETURN scalar_expression
END
```
Подробнее о функциях T-SQL:
https://docs.microsoft.com/ru-ru/sql/t-sql/statements/create-function-transact-sql?view=sql-server-2017

Пример #1:

In [1]:
use tempdb
go

if OBJECT_ID('dbo.fn_GetSwimmersCount', 'F') is not null
   drop function dbo.fn_GetSwimmersCount
go

-------------------------------------------------------------------------------------------
-- function returns number of Swimmers for given club (@ClubID)
-- created by:   Timofey Gavrilenko
-- created date: 4/26/2019
-- sample call:  
-- select dbo.fn_GetSwimmersCount(default)
-----------------------------------------------------------------------------------

create function dbo.fn_GetSwimmersCount(
    @ClubID int = null
) returns int
as  
begin
  return (select count(1) from dbo.Swimmer where SwimmingClubId = @ClubID or @ClubID is null)
end
go

Commands completed successfully.

Total execution time: 00:00:00.0015604

Commands completed successfully.

Total execution time: 00:00:00.0012716

Commands completed successfully.

Total execution time: 00:00:00.0164098

_Табличная функция_ возвращает набор данных (виртуальную таблицу)


Может быть:
- inline (лучше оптимизируются)
- multi-statement (больше возможностей)

Синтаксис:
```
-- Transact-SQL Inline Table-Valued Function Syntax   
CREATE [ OR ALTER ] FUNCTION [ schema_name. ] function_name   
( [ { @parameter_name [ AS ] [ type_schema_name. ] parameter_data_type   
    [ = default ] [ READONLY ] }   
    [ ,...n ]  
  ]  
)  
RETURNS TABLE  
    [ WITH <function_option> [ ,...n ] ]  
    [ AS ]  
    RETURN [ ( ] select_stmt [ ) ]  
[ ; ]
```
-----------------------------------------------------------------------

```
-- Transact-SQL Multi-Statement Table-Valued Function Syntax  
CREATE [ OR ALTER ] FUNCTION [ schema_name. ] function_name   
( [ { @parameter_name [ AS ] [ type_schema_name. ] parameter_data_type   
    [ = default ] [READONLY] }   
    [ ,...n ]  
  ]  
)  
RETURNS @return_variable TABLE <table_type_definition>  
    [ WITH <function_option> [ ,...n ] ]  
    [ AS ]  
    BEGIN   
        function_body   
        RETURN  
    END  
[ ; ]
```



Пример #2:

Пример #3: